In [1]:
import json
import yfinance as yf
import time

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

# retorna_cotacao_acao_historica

In [2]:
def retorna_cotacao_acao_historica(
        ticker,
        periodo='1mo'
):
    
    ticker = yf.Ticker(ticker)
    hist = ticker.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')  # Formata o índice do DataFrame hist para strings no formato 'AAAA-MM-DD'.
    hist = round(hist, 2)   
    
    # Verifica se o DataFrame tem mais de 30 registros. Se tiver,  calcula um slice_size (tamanho fatia) dividindo por 30.
    # E reduz tamanho pegando um registro a cada slice_size, começando do final e revertendo para manter na sequência original.
    
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
        
    return hist.to_json() # Passar como dicionario, e não como DF



# Mapeando função
funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}

# Tools

In [3]:
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',
            'description': 'Retorna a cotação diária histórica para uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]  # API aceita somente esses dias
                    }
                }
            }
        }
    }

]


# Gera Texto e  tool_calls

In [4]:
def gera_texto(mensagens):
    resposta = client.chat.completions.create(
        messages=mensagens,
        model='gpt-3.5-turbo-0125',
        tools=tools,
        tool_choice='auto',
        #stream=True,
    )
    
    tool_calls = resposta.choices[0].message.tool_calls

    if tool_calls:
        mensagens.append(resposta.choices[0].message)
        for tool_call in tool_calls:
            func_name = tool_call.function.name
            function_to_call = funcoes_disponiveis[func_name]
            func_args = json.loads(tool_call.function.arguments)
            func_return = function_to_call(**func_args)  # Passa todos os parametros do 'func_args', para a function_to_call
            mensagens.append({
                'tool_call_id': tool_call.id,
                'role': 'tool',
                'name': func_name,
                'content': func_return
            })
            
        segunda_resposta = client.chat.completions.create(  #Capturando a resposta final do modelo, depois de utilizar a funcao
            messages=mensagens,
            model='gpt-3.5-turbo-0125',
        )
        mensagens.append(segunda_resposta.choices[0].message)
    
    print("Assistant: ", end='')
          
    for caractere in mensagens[-1].content:
        print(caractere, end='', flush=True)  # Impede que a saída seja armazenada em buffer
        time.sleep(0.1)
          
    #print(mensagens)
    #mensagemm = mensagens[-1].content
    
    #print(f'Assistant: {mensagens[-1].content}') # Imprimindo a resposta

    return mensagens

# Geração Chat

In [5]:
if __name__ == '__main__':

    print('Bem-vindo ao ChatBot Financeiro da Asimov.')

    while True:
        input_usuario = input('User: ')
        mensagens = [{'role': 'user', 'content': input_usuario}]
        mensagens = gera_texto(mensagens)

Bem-vindo ao ChatBot Financeiro da Asimov.
User: cotação atual PETR4
Assistant: A cotação atual da ação PETR4 é de R$ 36,61.


KeyboardInterrupt



In [ ]:
ticker = yf.Ticker("PETR4.SA")

In [ ]:
ticker.dividends

In [ ]:
ticker.splits

In [ ]:
ticker.news